In [2]:
!pip install prettytable

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\91739\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from prettytable import PrettyTable

# Load dataset (assuming it's already loaded as 'df')
df = pd.read_csv(r"restaurant_data.csv")

# Select features for clustering
features = ['Rating', 'Average Cost for Two (INR)', 'Total Reviews']
scaler = StandardScaler()

# Standardize the features
scaled_features = scaler.fit_transform(df[features])

# Apply K-Means clustering to the scaled features
kmeans = KMeans(n_clusters=3, random_state=42)  # You can adjust n_clusters
df['Cluster'] = kmeans.fit_predict(scaled_features)

# Function to recommend restaurants based on city, rating, price constraints, and optional cluster
def recommend_restaurants_by_city_and_constraints(user_city, min_rating, max_price, user_cluster=None):
    user_city = user_city.lower()

    # Check available cities in the dataset
    available_cities = df['City'].str.lower().unique()
    print(f"Available cities: {', '.join(available_cities)}")

    # Check if the entered city exists in the dataset
    if user_city not in available_cities:
        print(f"Sorry, we couldn't find any restaurants in the city '{user_city.capitalize()}'.")
        return pd.DataFrame()

    # Filter restaurants in the specified city
    city_restaurants = df[df['City'].str.lower() == user_city]
    print(f"Restaurants in {user_city.capitalize()}:")
    print(city_restaurants.head())

    # Apply cluster filtering if user_cluster is specified
    if user_cluster is not None:
        # Check if the cluster number exists in the dataset
        available_clusters = df['Cluster'].unique()
        print(f"Available clusters: {available_clusters}")
        
        if user_cluster not in available_clusters:
            print(f"Cluster {user_cluster} not found.")
            return pd.DataFrame()

        city_restaurants = city_restaurants[city_restaurants['Cluster'] == user_cluster]
    
    # Apply rating and price constraints
    filtered_restaurants = city_restaurants[
        (city_restaurants['Rating'] >= min_rating) &
        (city_restaurants['Average Cost for Two (INR)'] <= max_price)
    ]
    print(f"Filtered restaurants by rating >= {min_rating} and price <= {max_price}:")
    print(filtered_restaurants.head())

    return filtered_restaurants

# Get user input for city, rating, price, and optionally cluster
available_cities = df['City'].unique()
print(f"Available cities: {', '.join(available_cities)}")

user_city = input("Enter the city: ").strip().lower()
min_rating = float(input("Enter minimum rating (e.g., 4.0): "))
max_price = float(input("Enter maximum price for two (e.g., 1000): "))

# Optionally get user input for a specific cluster (if they want to filter by cluster)
user_cluster = input("Enter cluster number (optional, press Enter to skip): ").strip()
user_cluster = int(user_cluster) if user_cluster else None

# Recommend restaurants based on city, rating, price, and optional cluster constraints
recommended_restaurants = recommend_restaurants_by_city_and_constraints(user_city, min_rating, max_price, user_cluster)

# Display the recommended restaurants using PrettyTable if there are results
if recommended_restaurants.empty:
    print("No Restaurants found with the specified criteria.")
else:
    # Create a PrettyTable object to display the output in a tabular format
    table = PrettyTable()
    table.field_names = ["RestaurantID", "Name", "Location", "City", "Cuisine", "Rating", "Total Reviews", "Average Cost for Two (INR)", "Cluster"]

    # Add rows to the table
    for _, row in recommended_restaurants.iterrows():
        table.add_row([row['RestaurantID'], row['Name'], row['Location'], row['City'], row['Cuisine'], row['Rating'], row['Total Reviews'], row['Average Cost for Two (INR)'], row['Cluster']])

    # Print the table
    print(f"\nRecommended Restaurants in {user_city.capitalize()} with rating >= {min_rating} and price <= {max_price}:")
    print(table)



Available cities: Bangalore, Lucknow, Hyderabad, Jaipur, Pune, Kolkata, Chennai, Ahmedabad, Delhi, Mumbai
Available cities: bangalore, lucknow, hyderabad, jaipur, pune, kolkata, chennai, ahmedabad, delhi, mumbai
Restaurants in Delhi:
   RestaurantID              Name       Location   City   Cuisine  Rating  \
12      RID0013  Flavors of India      Salt Lake  Delhi  American     1.2   
48      RID0049       Noodle Nook        MG Road  Delhi   Chinese     1.9   
50      RID0051        Taco Tower     SG Highway  Delhi  Japanese     1.7   
60      RID0061        Thai Treat     Hazratganj  Delhi   Chinese     2.3   
86      RID0087    Spice Symphony  Banjara Hills  Delhi      Thai     3.2   

   Price Range  Total Reviews  Average Cost for Two (INR)  Cluster  
12         Low           2264                        1447        0  
48      Medium           3453                        3480        0  
50         Low           2924                        4598        0  
60         Low            5

In [7]:
!pip install graphviz

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\91739\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
from graphviz import Digraph

# Create a directed graph
pipeline = Digraph('ML_Pipeline', format='png', graph_attr={'rankdir': 'LR'})

# Add nodes
pipeline.node('A', 'Data Input\n(Raw Data)')
pipeline.node('B', 'Data Preprocessing\n(Cleaning, Scaling)')
pipeline.node('C', 'Model Training\n(ML Algorithm)')
pipeline.node('D', 'Model Evaluation\n(Metrics)')
pipeline.node('E', 'Dockerization\n(Containerize)')
pipeline.node('F', 'Jenkins Automation\n(CI/CD)')

# Connect nodes
pipeline.edges(['AB', 'BC', 'CD', 'DE', 'EF'])

# Save and render
pipeline.render('ml_pipeline', view=True)

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH